In [1]:
%matplotlib ipympl
from matplotlib import pylab as plt
import matplotlib
import molpx

import os
import sys
import glob
from itertools import cycle

import numpy as np
import pandas as pd

from os.path import join

import seaborn as sns

import mdtraj as md
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform

import pyemma

import random

from os.path import join

from collections import OrderedDict
from collections import defaultdict


print("Python version:\n{}\n".format(sys.version))
print("matplotlib version: {}".format(matplotlib.__version__))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("seaborn version: {}".format(sns.__version__))

Python version:
3.6.5 |Anaconda custom (64-bit)| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]

matplotlib version: 2.2.2
pandas version: 0.23.0
numpy version: 1.12.1
seaborn version: 0.8.1


In [2]:
topfile = './Trajectories/open10_dry.parm7'
from glob import glob
trajfile = 'toy_traj.nc'

In [3]:
ref1 = md.load('./Trajectories/open10_dry.rst7', top='./Trajectories/open10_dry.parm7')
ref2 = md.load('./Trajectories/closed10_dry.rst7', top='./Trajectories/closed10_dry.parm7')

In [4]:
feat = pyemma.coordinates.featurizer(topfile)
feat.add_minrmsd_to_ref(ref1)
feat.add_minrmsd_to_ref(ref2)
feat.describe()

['minrmsd to frame 0 of <mdtraj.Trajectory with 1 frames, 2808 atoms, 192 residues, and unitcells at 0x7fab11ca30b8>',
 'minrmsd to frame 0 of <mdtraj.Trajectory with 1 frames, 2808 atoms, 192 residues, and unitcells at 0x7fab115d00b8>']

In [5]:
feat.dimension()

2

In [12]:
inp = pyemma.coordinates.load(trajfile, stride=100, features=feat)

In [1]:
pyemma.plots.plot_free_energy(np.concatenate(inp)[:0], np.concatenate(inp)[:1])
plt.xlabel('RMSD to Open') 
plt.ylabel('RMSD to Closed');

NameError: name 'pyemma' is not defined

In [14]:
inp

array([[ 0.3915436 ,  0.20261499],
       [ 0.38974261,  0.21436335],
       [ 0.38427576,  0.24876583],
       ..., 
       [ 0.35312277,  0.40201953],
       [ 0.33655626,  0.36866042],
       [ 0.33706945,  0.37446961]], dtype=float32)

In [16]:
print('number of trajectories = ',inp.number_of_trajectories())
print('trajectory length = ',inp.trajectory_length(0), 'frames')
print('trajectory time = ',(inp.trajectory_length(0)*50/1000),'ns')
print('trajectory time step = ',(inp.trajectory_length(0)*50)/inp.trajectory_length(0),'ps')
print('number of dimension = ',inp.dimension())

AttributeError: 'numpy.ndarray' object has no attribute 'number_of_trajectories'

In [10]:
inp

In [17]:
tica_obj = pyemma.coordinates.tica(data=inp, lag=20, var_cutoff=0.95)
print('TICA dimension ', tica_obj.dimension())

TICA dimension  2


In [18]:
tica_obj.cumvar

array([ 0.70874354,  1.        ])

In [19]:
Y = tica_obj.get_output()

In [20]:
t = md.load(trajfile, top=topfile, stride=100)
t

<mdtraj.Trajectory with 3964 frames, 2808 atoms, 192 residues, and unitcells at 0x7fab117495f8>

In [21]:
mpx_wdg_box = molpx.visualize.FES(t,
                                 #MD_trajfiles,
                                 t.topology,
                                 Y,
                                 #Y,
                                 nbins=50,
                                 #proj_idxs=[1,2],
                                 proj_labels='TIC',
                                 #n_overlays=5,
                                )
mpx_wdg_box


MolPXHBox(children=(NGLWidget(count=95), FigureCanvasNbAgg()))

In [22]:
matplotlib.rcParams.update({'font.size': 14})
dt = 0.01
plt.figure(figsize=(10, 10))
ax1=plt.subplot(211)
x = dt*np.arange(Y[0].shape[0])
plt.plot(x, Y[0][:, 0], linestyle='None', marker='o', markersize=.2); plt.ylabel('IC 1'); plt.xticks([]);
ax1=plt.subplot(212)
plt.plot(x, Y[0][:, 1], linestyle='None', marker='o', markersize=.2); plt.ylabel('IC 2'); plt.xticks([]);  

FigureCanvasNbAgg()

In [18]:
matplotlib.rcParams.update({'font.size': 14})
dt = 0.01
plt.figure(figsize=(10, 10))
ax1=plt.subplot(211)
x = dt*np.arange(Y[0].shape[0])
plt.plot(x, Y[0][:, 0], linestyle='None', marker='o', markersize=.2); plt.ylabel('IC 1'); plt.xticks([]); plt.yticks(np.arange(-4, 4, 1))
ax1=plt.subplot(212)
plt.plot(x, Y[0][:, 1], linestyle='None', marker='o', markersize=.2); plt.ylabel('IC 2'); plt.xticks([]);  plt.yticks(np.arange(-4, 4, 1))
plt.show()

FigureCanvasNbAgg()